In [3]:
import matplotlib.pyplot as plt
import numpy as np
%run activators.ipynb
%run cnn.ipynb

[[[ 6.  7.  5.]
  [ 3. -1. -1.]
  [ 2. -1.  4.]]

 [[ 2. -5. -8.]
  [ 1. -4. -4.]
  [ 0. -5. -5.]]]
filter weights:
array([[[-1.008,  0.99 , -0.009],
        [-0.005,  0.994, -0.006],
        [-0.006,  0.995,  0.996]],

       [[-1.004, -1.001, -0.004],
        [-0.01 , -0.009, -0.012],
        [-0.002, -1.002, -0.002]],

       [[-0.002, -0.002, -1.003],
        [-0.005,  0.992, -0.005],
        [ 0.993, -1.008, -1.007]]])
bias:
0.991
filter weights:
array([[[ 9.980e-01,  9.980e-01, -1.001e+00],
        [-1.004e+00, -1.007e+00,  9.970e-01],
        [-4.000e-03, -1.004e+00,  9.980e-01]],

       [[ 0.000e+00,  9.990e-01,  0.000e+00],
        [-1.009e+00, -5.000e-03, -1.004e+00],
        [-1.004e+00,  1.000e+00,  0.000e+00]],

       [[-1.004e+00, -6.000e-03, -5.000e-03],
        [-1.002e+00, -5.000e-03,  9.980e-01],
        [-1.002e+00, -1.000e-03,  0.000e+00]]])
bias:
-0.007
weights(0,0,0): expected - actural 5.000000 - 5.000000
weights(0,0,1): expected - actural 6.000000 - 6.000000
w

In [4]:
class LstmLayer(object):
    def __init__(self,input_width,state_width,learning_rate):
        self.input_width = input_width
        self.state_width = state_width
        self.learning_rate = learning_rate
         # 门的激活函数
        self.gate_activator = SigmoidActivator()
        # 输出的激活函数
        self.output_activator = TanhActivator()
        # 当前时刻初始化为t0
        self.times = 0       
        # 各个时刻的单元状态向量c
        self.c_list = self.init_state_vec()
        # 各个时刻的输出向量h
        self.h_list = self.init_state_vec()
        # 各个时刻的遗忘门f
        self.f_list = self.init_state_vec()
        # 各个时刻的输入门i
        self.i_list = self.init_state_vec()
        # 各个时刻的输出门o
        self.o_list = self.init_state_vec()
        # 各个时刻的即时状态c~
        self.ct_list = self.init_state_vec()
        # 遗忘门权重矩阵Wfh, Wfx, 偏置项bf
        self.Wfh, self.Wfx, self.bf = (self.init_weight_mat())
        # 输入门权重矩阵Wfh, Wfx, 偏置项bf
        self.Wih, self.Wix, self.bi = (self.init_weight_mat())
        # 输出门权重矩阵Wfh, Wfx, 偏置项bf
        self.Woh, self.Wox, self.bo = (self.init_weight_mat())
        # 单元状态权重矩阵Wfh, Wfx, 偏置项bf
        self.Wch, self.Wcx, self.bc = (self.init_weight_mat())
        
    # 初始化保存状态的向量
    def init_state_vec(self):
        state_vec_list = []
        state_vec_list.append(np.zeros((self.state_width, 1)))
        return state_vec_list
    
    # 初始化权重矩阵
    def init_weight_mat(self):
        Wh = np.random.uniform(-1e-4, 1e-4,(self.state_width, self.state_width))
        Wx = np.random.uniform(-1e-4, 1e-4,(self.state_width, self.input_width))
        b = np.zeros((self.state_width, 1))
        return Wh, Wx, b
    
    # 前向计算
    def forward(self, x):
        self.times += 1
        # 遗忘门
        fg = self.calc_gate(x, self.Wfx, self.Wfh, self.bf, self.gate_activator)
        self.f_list.append(fg)
        # 输入门
        ig = self.calc_gate(x, self.Wix, self.Wih,self.bi, self.gate_activator)
        self.i_list.append(ig)
        # 输出门
        og = self.calc_gate(x, self.Wox, self.Woh,self.bo, self.gate_activator)
        self.o_list.append(og)
        # 即时状态
        ct = self.calc_gate(x, self.Wcx, self.Wch,self.bc, self.output_activator)
        self.ct_list.append(ct)
        # 单元状态
        c = fg * self.c_list[self.times - 1] + ig * ct
        self.c_list.append(c)
        # 输出
        h = og * self.output_activator.forward(c)
        self.h_list.append(h)
        
    # 计算门
    def calc_gate(self,x,Wx,Wh,b,activator):
        h=self.h_list[self.times-1] # 上次的LSTM输出
        net = np.dot(Wh,h)+np.dot(Wx,x)+b
        gate=activator.forward(net)
        return gate
    
    # LSTM训练算法
    def backward(self,x,delta_h,activator):
        self.calc_delta(delta_h,activator)
        self.calc_gradient(x)
        
    # 按照梯度下降，更新权重
    def update(self):
        self.Wfh -= self.learning_rate * self.Whf_grad
        self.Wfx -= self.learning_rate * self.Whx_grad
        self.bf -= self.learning_rate * self.bf_grad
        self.Wih -= self.learning_rate * self.Whi_grad
        self.Wix -= self.learning_rate * self.Whi_grad
        self.bi -= self.learning_rate * self.bi_grad
        self.Woh -= self.learning_rate * self.Wof_grad
        self.Wox -= self.learning_rate * self.Wox_grad
        self.bo -= self.learning_rate * self.bo_grad
        self.Wch -= self.learning_rate * self.Wcf_grad
        self.Wcx -= self.learning_rate * self.Wcx_grad
        self.bc -= self.learning_rate * self.bc_grad
        
    def calc_delta(self,delta_h,activator):
        # 初始化各个时刻的误差项
        self.delta_h_list = self.init_delta()  # 输出误差项
        self.delta_o_list = self.init_delta()  # 输出门误差项
        self.delta_i_list = self.init_delta()  # 输入门误差项
        self.delta_f_list = self.init_delta()  # 遗忘门误差项
        self.delta_ct_list = self.init_delta() # 即时输出误差项

        # 保存从上一层传递下来的当前时刻的误差项
        self.delta_h_list[-1] = delta_h
        
        # 迭代计算每个时刻的误差项
        for k in range(self.times, 0, -1):
            self.calc_delta_k(k)
    
    # 初始化误差项
    def init_delta(self):
        delta_list=[]
        for i in range(self.time+1):
            delta_list.append(np.zeros((self.state_width,1)))
        return delta_list
    
    # 根据k时刻的delta_h，计算k时刻的delta_f、delta_i、delta_o、delta_ct，以及k-1时刻的delta_h
    def calc_delta_k(self,k):
        # 获得k时刻前向计算的值
        ig = self.i_list[k]
        og = self.o_list[k]
        fg = self.f_list[k]
        ct = self.ct_list[k]
        c = self.c_list[k]
        c_prev = self.c_list[k-1]
        tanh_c = self.output_activator.forward(c)
        delta_k = self.delta_h_list[k]
        
        delta_o = (delta_k * tanh_c * self.gate_activator.backward(og))
        delta_f = (delta_k * og * (1 - tanh_c * tanh_c) * c_prev * self.gate_activator.backward(fg))
        delta_i = (delta_k * og * (1 - tanh_c * tanh_c) * ct * self.gate_activator.backward(ig))
        delta_ct = (delta_k * og * (1 - tanh_c * tanh_c) * ig * self.output_activator.backward(ct))
        delta_h_prev = (
                np.dot(delta_o.transpose(), self.Woh) +
                np.dot(delta_i.transpose(), self.Wih) +
                np.dot(delta_f.transpose(), self.Wfh) +
                np.dot(delta_ct.transpose(), self.Wch)
            ).transpose()

        # 保存全部delta值
        self.delta_h_list[k-1] = delta_h_prev
        self.delta_f_list[k] = delta_f
        self.delta_i_list[k] = delta_i
        self.delta_o_list[k] = delta_o
        self.delta_ct_list[k] = delta_ct
        
    def calc_gradient(self, x):
        # 初始化遗忘门权重梯度矩阵和偏置项
        self.Wfh_grad, self.Wfx_grad, self.bf_grad = (self.init_weight_gradient_mat())
        # 初始化输入门权重梯度矩阵和偏置项
        self.Wih_grad, self.Wix_grad, self.bi_grad = (self.init_weight_gradient_mat())
        # 初始化输出门权重梯度矩阵和偏置项
        self.Woh_grad, self.Wox_grad, self.bo_grad = (self.init_weight_gradient_mat())
        # 初始化单元状态权重梯度矩阵和偏置项
        self.Wch_grad, self.Wcx_grad, self.bc_grad = (self.init_weight_gradient_mat())
        
         # 计算对上一次输出h和偏置项b的权重梯度
        for t in range(self.times, 0, -1):
            # 计算各个时刻的梯度
            (Wfh_grad, bf_grad,
            Wih_grad, bi_grad,
            Woh_grad, bo_grad,
            Wch_grad, bc_grad) = (self.calc_gradient_t(t))
            # 实际梯度是各时刻梯度之和
            self.Wfh_grad += Wfh_grad
            self.bf_grad += bf_grad
            self.Wih_grad += Wih_grad
            self.bi_grad += bi_grad
            self.Woh_grad += Woh_grad
            self.bo_grad += bo_grad
            self.Wch_grad += Wch_grad
            self.bc_grad += bc_grad
            
        # 计算对本次输入x的权重梯度
        xt = x.transpose()
        self.Wfx_grad = np.dot(self.delta_f_list[-1], xt)
        self.Wix_grad = np.dot(self.delta_i_list[-1], xt)
        self.Wox_grad = np.dot(self.delta_o_list[-1], xt)
        self.Wcx_grad = np.dot(self.delta_ct_list[-1], xt)
    
    # 初始化权重矩阵
    def init_weight_gradient_mat(self):
        Wh_grad = np.zeros((self.state_width,self.state_width))
        Wx_grad = np.zeros((self.state_width,self.input_width))
        b_grad = np.zeros((self.state_width, 1))
        return Wh_grad, Wx_grad, b_grad
    
    # 计算每个时刻t权重的梯度
    def calc_gradient_t(self, t):
        h_prev = self.h_list[t-1].transpose()
        Wfh_grad = np.dot(self.delta_f_list[t], h_prev)
        bf_grad = self.delta_f_list[t]
        Wih_grad = np.dot(self.delta_i_list[t], h_prev)
        bi_grad = self.delta_f_list[t]
        Woh_grad = np.dot(self.delta_o_list[t], h_prev)
        bo_grad = self.delta_f_list[t]
        Wch_grad = np.dot(self.delta_ct_list[t], h_prev)
        bc_grad = self.delta_ct_list[t]
        return Wfh_grad, bf_grad, Wih_grad, bi_grad, Woh_grad, bo_grad, Wch_grad, bc_grad
    
    def reset_state(self):
        # 当前时刻初始化为t0
        self.times = 0       
        # 各个时刻的单元状态向量c
        self.c_list = self.init_state_vec()
        # 各个时刻的输出向量h
        self.h_list = self.init_state_vec()
        # 各个时刻的遗忘门f
        self.f_list = self.init_state_vec()
        # 各个时刻的输入门i
        self.i_list = self.init_state_vec()
        # 各个时刻的输出门o
        self.o_list = self.init_state_vec()
        # 各个时刻的即时状态c~
        self.ct_list = self.init_state_vec()

def data_set():
    x = [np.array([[1], [2], [3]]),
         np.array([[2], [3], [4]])]
    d = np.array([[1], [2]])
    return x, d

# 梯度检查
def gradient_check():
    # 设计一个误差函数，取所有节点输出项之和
    error_function = lambda o: o.sum()
    
    lstm = LstmLayer(3, 2, 1e-3)

    # 计算forward值
    x, d = data_set()
    lstm.forward(x[0])
    lstm.forward(x[1])
    
    # 求取sensitivity map
    sensitivity_array = np.ones(lstm.h_list[-1].shape,dtype=np.float64)
    # 计算梯度
    lstm.backward(x[1], sensitivity_array, IdentityActivator())
    
    # 检查梯度
    epsilon = 10e-4
    for i in range(lstm.Wfh.shape[0]):
        for j in range(lstm.Wfh.shape[1]):
            lstm.Wfh[i,j] += epsilon
            lstm.reset_state()
            lstm.forward(x[0])
            lstm.forward(x[1])
            err1 = error_function(lstm.h_list[-1])
            lstm.Wfh[i,j] -= 2*epsilon
            lstm.reset_state()
            lstm.forward(x[0])
            lstm.forward(x[1])
            err2 = error_function(lstm.h_list[-1])
            expect_grad = (err1 - err2) / (2 * epsilon)
            lstm.Wfh[i,j] += epsilon
            print('weights(%d,%d): expected - actural %.4e - %.4e' % (i, j, expect_grad, lstm.Wfh_grad[i,j]))
    return lstm

def test():
    l = LstmLayer(3, 2, 1e-3)
    x, d = data_set()
    l.forward(x[0])
    l.forward(x[1])
    l.backward(x[1], d, IdentityActivator())
    return l

In [5]:
if __name__ == '__main__':
    test()
    gradient_check()

AttributeError: 'LstmLayer' object has no attribute 'time'

为什么初始化误差时，delta_list的times要加一，而初始化梯度时不需要?
    在循环神经网络（RNN）和特别是LSTM（长短期记忆）网络中，delta_list 通常用于存储每个时间步的隐藏状态 h 和细胞状态 c 的梯度（或称为敏感度）。在初始化这些梯度时，delta_list 的长度通常与输入序列的长度（时间步的数量）相同。
    当我们谈论初始化误差时，delta_list 的长度加一的原因通常与网络的输出层有关。在序列到序列（seq2seq）模型或带有输出层的RNN中，最后一个时间步的输出通常用于计算损失函数（如交叉熵损失）。在计算损失函数关于模型参数的梯度时，我们需要考虑输出层的贡献。
    因此，delta_list 的最后一个元素（对应于输出层的梯度）是在损失函数计算后单独计算的，而不是通过时间反向传播（Backpropagation Through Time, BPTT）得到的。
    然而，当我们初始化梯度时，我们是在为BPTT做准备，所以我们只关心隐藏状态和细胞状态的梯度，这些梯度是通过时间步反向传播的。因此，初始化梯度时，我们不需要为输出层额外添加一个元素。
简而言之：
    初始化误差（delta_list）时，我们需要为输出层的梯度额外添加一个元素，因此长度加一。
    初始化梯度时，我们只关心隐藏状态和细胞状态的梯度，所以不需要额外添加元素。
    需要注意的是，具体的实现细节可能会根据网络架构和所使用的库而有所不同。在一些实现中，可能不需要显式地初始化 delta_list，因为库会自动处理这些计算。然而，了解这些基本概念对于理解RNN和LSTM的训练过程是非常重要的。

为什么delta_h的初始值为sensitivity_array = np.ones(lstm.h_list[-1].shape, dtype=np.float64)?
    在LSTM（长短期记忆）网络中，delta_h 通常指的是隐藏状态 h 的梯度（或敏感度），这些梯度在反向传播过程中用于更新网络的权重。初始化 delta_h 为一个与隐藏状态形状相同的全1数组 sensitivity_array 的原因通常与反向传播算法和梯度计算有关。
    首先，让我们回顾一下反向传播算法的基本思想:
    在反向传播过程中，我们需要计算损失函数对于网络参数的梯度，以便使用这些梯度来更新参数并减少模型的预测错误。对于循环神经网络（RNNs）和特别是LSTM，这涉及到计算损失函数对于每个时间步的隐藏状态和细胞状态的梯度。在LSTM中，每个时间步的隐藏状态 h 和细胞状态 c 是根据前一个时间步的隐藏状态和输入来计算的。因此，在计算梯度时，我们需要知道每个时间步的隐藏状态和细胞状态如何影响最终的损失。
    初始化 delta_h 为全1数组的原因是为了在反向传播开始时为所有隐藏状态提供相同的初始敏感度。这意味着一开始时，我们假设每个隐藏状态对损失函数的影响都是相同的。随着反向传播的进行，delta_h 的值会根据链式法则和网络的权重进行更新，以反映每个隐藏状态对最终损失的实际影响。
    此外，使用全1数组作为初始值可以简化计算，因为所有的隐藏状态在开始时都有相同的敏感度，这有助于避免在计算梯度时引入额外的偏差。随着反向传播的进行，这些敏感度会根据网络的实际结构和权重进行调整。
    总的来说，将 delta_h 初始化为全1数组是反向传播算法的一部分，用于在开始时为所有隐藏状态提供相同的敏感度，并在后续的计算中根据网络的实际情况进行更新。